In [ ]:
#!pip install safetensors huggingface_hub safetensors transformers accelerate

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-c

In [ ]:
!git config --global credential.helper store

In [ ]:
import torch
from safetensors import safe_open
from safetensors.torch import save_file
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dfurman/llama-7b-fp16")

model = AutoModelForCausalLM.from_pretrained("dfurman/llama-7b-fp16",
                                             torch_dtype=torch.bfloat16)
model

In [ ]:
#!mkdir llama_7b
#model.save_pretrained('/content/llama_7b')
#tokenizer.save_pretrained('/content/llama_7b')

In [ ]:
# iterate over each bin file in model folder

tensor = torch.load("/content/llama_7b/pytorch_model-00002-of-00002.bin")
tensor

In [ ]:
save_file(tensor, "/content/llama_7b/model-00002-of-00002.safetensors")


In [ ]:

tensors = {}
with safe_open("/content/llama_7b/model-00002-of-00002.safetensors", framework="pt", device="cpu") as f:
   for key in f.keys():
       tensors[key] = f.get_tensor(key)

tensors

In [ ]:
tensors

In [ ]:
model_name = "/content/llama_7b"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = transformers.LlamaForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)

print(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [ ]:
model.push_to_hub("dfurman/llama-7b-fp16", use_auth_token=True)
tokenizer.push_to_hub("dfurman/llama-7b-fp16", use_auth_token=True)

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dfurman/llama-7b-fp16/commit/802d91c6f500a4aed33e44abe52cf6c8ca12a870', commit_message='Upload tokenizer', commit_description='', oid='802d91c6f500a4aed33e44abe52cf6c8ca12a870', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!git clone https://huggingface.co/dfurman/llama-7b-fp16

Cloning into 'llama-7b-fp16'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 32 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), 488.62 KiB | 1.87 MiB/s, done.
Filtering content: 100% (2/2), 4.55 GiB | 20.65 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
!cp /content/llama_7b/model-00001-of-00002.safetensors /content/llama-7b-fp16
!cp /content/llama_7b/model-00002-of-00002.safetensors /content/llama-7b-fp16

In [ ]:
os.chdir('llama-7b-fp16')

In [ ]:
!ls

config.json			  pytorch_model-00002-of-00002.bin
generation_config.json		  pytorch_model.bin.index.json
LICENSE				  README.md
model-00001-of-00002.safetensors  special_tokens_map.json
model-00002-of-00002.safetensors  tokenizer_config.json
pytorch_model-00001-of-00002.bin  tokenizer.json


In [ ]:
!git config --global user.email "dryanfurman@gmail.com"

In [ ]:
!cp /content/llama-7b-fp16/model-00001-of-00002.safetensors /content/drive/MyDrive/llama_7b_safetensors/model-00001-of-00002.safetensors

In [ ]:
!cp /content/llama-7b-fp16/model-00002-of-00002.safetensors /content/drive/MyDrive/llama_7b_safetensors/model-00002-of-00002.safetensors

In [ ]:
#!git add .
#!git commit -m "safetensors added"
#!git push

fatal: could not read Username for 'https://huggingface.co': No such device or address


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dfurman/llama-7b-fp16")

model = AutoModelForCausalLM.from_pretrained("dfurman/llama-7b-fp16",
                                             torch_dtype=torch.float16)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 6>:6                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:484 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   481 │   │   │   )                                                                              │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   483 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 484 │   │   │   return model_class.from_pretrained(                                            │
│   485 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   486 │   │   │   )                                                                              │
│   487 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2585 in from_pretrained   │
│                                                                                                  │
│   2582 │   │   # We'll need to download and cache each checkpoint shard if the checkpoint is sh  │
│   2583 │   │   if is_sharded:                                                                    │
│   2584 │   │   │   # rsolved_archive_file becomes a list of files that point to the different c  │
│ ❱ 2585 │   │   │   resolved_archive_file, sharded_metadata = get_checkpoint_shard_files(         │
│   2586 │   │   │   │   pretrained_model_name_or_path,                                            │
│   2587 │   │   │   │   resolved_archive_file,                                                    │
│   2588 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:958 in                         │
│ get_checkpoint_shard_files                                                                       │
│                                                                                                  │
│    955 │   for shard_filename in tqdm(shard_filenames, desc="Downloading shards", disable=not s  │
│    956 │   │   try:                                                                              │
│    957 │   │   │   # Load from URL                                                               │
│ ❱  958 │   │   │   cached_filename = cached_file(                                                │
│    959 │   │   │   │   pretrained_model_name_or_path,                                            │
│    960 │   │   │   │   shard_filename,                                                           │
│    961 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:417 in cached_file             │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached  

In [ ]:
model

In [ ]:

inputs = tokenizer("Paris is the capital of", return_tensors="pt", return_token_type_ids=False)

# Example 1: Print the scores for each token generated with Greedy Search
outputs = model.generate(**inputs, max_new_tokens=100)
tokenizer.decode(outputs[0], skip_special_tokens=True)